In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [3]:
# Loading Model
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D227F36000>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D227F6CE60>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
# Response from llm
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Pritam Saha and I am CS graduate.")])

AIMessage(content="Hello Pritam Saha, it's nice to meet you! Congratulations on graduating with a CS degree. \n\nWhat are you interested in doing now that you've graduated?  Are you looking for a job, continuing your education, or exploring other opportunities?\n\nI'm happy to help in any way I can, whether it's answering questions about technology, brainstorming career ideas, or just having a conversation.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 23, 'total_tokens': 111, 'completion_time': 0.16, 'prompt_time': 0.001351149, 'queue_time': 0.248000751, 'total_time': 0.161351149}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c3a68ecb-4e87-4908-b730-a012281c2d3c-0', usage_metadata={'input_tokens': 23, 'output_tokens': 88, 'total_tokens': 111})

In [5]:
# 
from re import A
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, my name is Pritam Saha and I am CS graduate."),
    AIMessage(content="Hello Pritam! How can I assist you today?"),
    HumanMessage(content="Can you tell me my name & what do I do?")
])

AIMessage(content='You said your name is Pritam Saha and that you are a CS graduate. \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 56, 'total_tokens': 87, 'completion_time': 0.056363636, 'prompt_time': 0.002191938, 'queue_time': 0.24509051, 'total_time': 0.058555574}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c5122d79-4724-45b8-acf3-7108d9c4c5a3-0', usage_metadata={'input_tokens': 56, 'output_tokens': 31, 'total_tokens': 87})

# Message History

In [6]:
# Seperate one chat session from others
from turtle import st
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [7]:
# Interact with the LLM based on the Chat History

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [8]:
#
config={"configurable": {"session_id": "chat1"}}

In [9]:
# Interat based on the session id

response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name is what?")],
    config = config,
)

response

AIMessage(content="As an AI, I don't have access to past conversations. Can you tell me your name? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 0.001259528, 'queue_time': 0.244403592, 'total_time': 0.046714073}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4785c7b3-7eb0-4593-8d0b-311fd2b59410-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41})

In [10]:
# changing the session Id to test 

config1={"configurable": {"session_id": "chat2"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name is what?")],
    config= config1,
)

response.content

"As an AI, I don't have access to past conversations or any information about you. So, I don't know your name.\n\nWhat's your name? 😊\n"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, do you know my name ?")],
    config= config1,
)

response.content

"As a large language model, I have no memory of past conversations. So, I don't know your name. \n\nIf you'd like to tell me, I'd be happy to know! 😊  \n\n"

# Prompt Template

In [12]:
from click import prompt
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that helps people find the answers."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [13]:
chain.invoke({"messages":[HumanMessage(content="Hi, my name is Pritam Saha")]})

AIMessage(content="Hi Pritam Saha, it's nice to meet you! \n\nI'm ready to help you find the answers you need. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 29, 'total_tokens': 71, 'completion_time': 0.076363636, 'prompt_time': 0.001402469, 'queue_time': 0.24780896, 'total_time': 0.077766105}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a6287758-d28e-43f5-b8c6-bc290f28259c-0', usage_metadata={'input_tokens': 29, 'output_tokens': 42, 'total_tokens': 71})

In [14]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [15]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Pritam")],
    config=config
)

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="Can you tell me what is my name & what do I do?")],
    config=config
)

response.content

"You told me your name is Pritam earlier!  \n\nI don't know what you do, though.  Would you like to tell me? 😊 \n\nRemember, I'm here to learn about you!  \n\n"

In [21]:
# Added more complexity


prompt= ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that helps people find the answers with best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model

In [22]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Pritam Saha")], "language": "Bengali"})

response.content

'নমস্কার Pritam Saha! \n\nআমি আপনার সাহায্য করার জন্য এখানে।  আপনি কি জানতে চান?  \n\nআপনার প্রশ্ন দিয়ে আমি যথাসম্ভব সঠিক উত্তর খুঁজে পাব।  😊\n\n'